In [1]:
import os
import os.path as ops
import argparse
import time
import math

import tensorflow as tf
import glob
import glog as log
import numpy as np
import matplotlib.pyplot as plt
import cv2

from lanenet_model import lanenet_merge_model
from lanenet_model import lanenet_cluster
from lanenet_model import lanenet_postprocess
from config import global_config

In [2]:
CFG = global_config.cfg
VGG_MEAN = [103.939, 116.779, 123.68]

In [3]:
CFG

{'TEST': {'BATCH_SIZE': 8,
  'GPU_MEMORY_FRACTION': 0.8,
  'TF_ALLOW_GROWTH': True},
 'TRAIN': {'BATCH_SIZE': 8,
  'CLASSES_NUMS': 2,
  'DISPLAY_STEP': 1,
  'EPOCHS': 200010,
  'GPU_MEMORY_FRACTION': 0.85,
  'IMG_HEIGHT': 256,
  'IMG_WIDTH': 512,
  'LEARNING_RATE': 0.0005,
  'LR_DECAY_RATE': 0.1,
  'LR_DECAY_STEPS': 410000,
  'MOMENTUM': 0.9,
  'TEST_DISPLAY_STEP': 1000,
  'TF_ALLOW_GROWTH': True,
  'VAL_BATCH_SIZE': 8}}

In [4]:
def minmax_scale(input_arr):
    """

    :param input_arr:
    :return:
    """
    min_val = np.min(input_arr)
    max_val = np.max(input_arr)

    output_arr = (input_arr - min_val) * 255.0 / (max_val - min_val)
    return output_arr

In [5]:
from lanenet_model import lanenet_merge_model
from lanenet_model import lanenet_cluster
from lanenet_model import lanenet_postprocess
def test_lanenet(image_path, weights_path, use_gpu):
    """

    :param image_path:
    :param weights_path:
    :param use_gpu:
    :return:
    """
    assert ops.exists(image_path), '{:s} not exist'.format(image_path)

    log.info('Start reading image data and pre-processing')
    t_start = time.time()
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image_vis = image
    image = cv2.resize(image, (512, 256), interpolation=cv2.INTER_LINEAR)
    image = image - VGG_MEAN
    log.info('Image is read, time consuming: {:.5f}s'.format(time.time() - t_start))

    input_tensor = tf.placeholder(dtype=tf.float32, shape=[1, 256, 512, 3], name='input_tensor')
    phase_tensor = tf.constant('test', tf.string)

    net = lanenet_merge_model.LaneNet(phase=phase_tensor, net_flag='vgg')
    binary_seg_ret, instance_seg_ret = net.inference(input_tensor=input_tensor, name='lanenet_model')

    cluster = lanenet_cluster.LaneNetCluster()
    postprocessor = lanenet_postprocess.LaneNetPoseProcessor()

    saver = tf.train.Saver()

    # Set sess configuration
    if use_gpu:
        sess_config = tf.ConfigProto(device_count={'GPU': 1})
    else:
        sess_config = tf.ConfigProto(device_count={'CPU': 0})
        
    sess_config.gpu_options.per_process_gpu_memory_fraction = CFG.TEST.GPU_MEMORY_FRACTION
    sess_config.gpu_options.allow_growth = CFG.TRAIN.TF_ALLOW_GROWTH
    sess_config.gpu_options.allocator_type = 'BFC'

    sess = tf.Session(config=sess_config)

    with sess.as_default():

        saver.restore(sess=sess, save_path=weights_path)

        t_start = time.time()
        binary_seg_image, instance_seg_image = sess.run([binary_seg_ret, instance_seg_ret],
                                                        feed_dict={input_tensor: [image]})
        t_cost = time.time() - t_start
        log.info('Single image lane line prediction time consuming: {:.5f}s'.format(t_cost))

        binary_seg_image[0] = postprocessor.postprocess(binary_seg_image[0])
        mask_image = cluster.get_lane_mask(binary_seg_ret=binary_seg_image[0],
                                           instance_seg_ret=instance_seg_image[0])

        for i in range(4):
            instance_seg_image[0][:, :, i] = minmax_scale(instance_seg_image[0][:, :, i])
        embedding_image = np.array(instance_seg_image[0], np.uint8)

        plt.figure('mask_image')
        plt.imshow(mask_image[:, :, (2, 1, 0)])
        plt.figure('src_image')
        plt.imshow(image_vis[:, :, (2, 1, 0)])
        plt.figure('instance_image')
        plt.imshow(embedding_image[:, :, (2, 1, 0)])
        plt.figure('binary_image')
        plt.imshow(binary_seg_image[0] * 255, cmap='gray')
        plt.show()

    sess.close()
    tf.reset_default_graph() 

return

In [6]:
def test_lanenet_batch(image_dir, weights_path, batch_size, use_gpu, save_dir=None):
    """

    :param image_dir:
    :param weights_path:
    :param batch_size:
    :param use_gpu:
    :param save_dir:
    :return:
    """
    assert ops.exists(image_dir), '{:s} not exist'.format(image_dir)

    log.info('Start getting the image file path...')
    image_path_list = glob.glob('{:s}/**/*.jpg'.format(image_dir), recursive=True) + \
                      glob.glob('{:s}/**/*.png'.format(image_dir), recursive=True) + \
                      glob.glob('{:s}/**/*.jpeg'.format(image_dir), recursive=True)

    input_tensor = tf.placeholder(dtype=tf.float32, shape=[None, 256, 512, 3], name='input_tensor')
    phase_tensor = tf.constant('test', tf.string)

    net = lanenet_merge_model.LaneNet(phase=phase_tensor, net_flag='vgg')
    binary_seg_ret, instance_seg_ret = net.inference(input_tensor=input_tensor, name='lanenet_model')

    cluster = lanenet_cluster.LaneNetCluster()
    postprocessor = lanenet_postprocess.LaneNetPoseProcessor()

    saver = tf.train.Saver()

    # Set sess configuration
    if use_gpu:
        sess_config = tf.ConfigProto(device_count={'GPU': 1})
    else:
        sess_config = tf.ConfigProto(device_count={'GPU': 0})
        
    sess_config.gpu_options.per_process_gpu_memory_fraction = CFG.TEST.GPU_MEMORY_FRACTION
    sess_config.gpu_options.allow_growth = CFG.TRAIN.TF_ALLOW_GROWTH
    sess_config.gpu_options.allocator_type = 'BFC'

    sess = tf.Session(config=sess_config)

    with sess.as_default():

        saver.restore(sess=sess, save_path=weights_path)

        epoch_nums = int(math.ceil(len(image_path_list) / batch_size))

        for epoch in range(epoch_nums):
            log.info('[Epoch:{:d}] Start image reading and preprocessing...'.format(epoch))
            
            t_start = time.time()
            image_path_epoch = image_path_list[epoch * batch_size:(epoch + 1) * batch_size]
            image_list_epoch = [cv2.imread(tmp, cv2.IMREAD_COLOR) for tmp in image_path_epoch]
            image_vis_list = image_list_epoch
            image_list_epoch = [cv2.resize(tmp, (512, 256), interpolation=cv2.INTER_LINEAR)
                                for tmp in image_list_epoch]
            image_list_epoch = [tmp - VGG_MEAN for tmp in image_list_epoch]
            t_cost = time.time() - t_start
            log.info('[Epoch:{:d}] Pre-processing {:d} images, total time consuming: {:.5f}s, \
            Average time per sheet: {:.5f}'.format(
                epoch, len(image_path_epoch), t_cost, t_cost / len(image_path_epoch)))

            t_start = time.time()
            binary_seg_images, instance_seg_images = sess.run(
                [binary_seg_ret, instance_seg_ret], feed_dict={input_tensor: image_list_epoch})
            t_cost = time.time() - t_start
            log.info('[Epoch:{:d}] Predicting {:d} image lane lines, total time: {:.5f}s, \
            average per time: {:.5f}s'.format(
                epoch, len(image_path_epoch), t_cost, t_cost / len(image_path_epoch)))

            cluster_time = []
            for index, binary_seg_image in enumerate(binary_seg_images):
                t_start = time.time()
                binary_seg_image = postprocessor.postprocess(binary_seg_image)
                mask_image = cluster.get_lane_mask(binary_seg_ret=binary_seg_image,
                                                   instance_seg_ret=instance_seg_images[index])
                cluster_time.append(time.time() - t_start)
                mask_image = cv2.resize(mask_image, (image_vis_list[index].shape[1],
                                                     image_vis_list[index].shape[0]),
                                        interpolation=cv2.INTER_LINEAR)

                if save_dir is None:
                    plt.ion()
                    plt.figure('mask_image')
                    plt.imshow(mask_image[:, :, (2, 1, 0)])
                    plt.figure('src_image')
                    plt.imshow(image_vis_list[index][:, :, (2, 1, 0)])
                    plt.pause(3.0)
                    plt.show()
                    plt.ioff()

                if save_dir is not None:
                    mask_image = cv2.addWeighted(image_vis_list[index], 1.0, mask_image, 1.0, 0)
                    image_name = ops.split(image_path_epoch[index])[1]
                    image_save_path = ops.join(save_dir, image_name)
                    cv2.imwrite(image_save_path, mask_image)

            log.info('[Epoch:{:d}] Perform {:d} image lane line clustering, total time: {:.5f}s, \
            average time per sheet: {:.5f}'.format(
                epoch, len(image_path_epoch), np.sum(cluster_time), np.mean(cluster_time)))

    sess.close()
    tf.reset_default_graph()

return

In [7]:
image_path = '/tf/network/lanenet-lane-detection/data/tusimple_test_image/'
weights_path = '/tf/network/lanenet-lane-detection/weights/tusimple_lanenet_vgg_2018-10-19-13-33-56.ckpt-200000'
is_batch = True
batch_size = 32
save_dir = '/tf/datasets/lanenet'
use_gpu = 1

if save_dir is not None and not ops.exists(save_dir):
    log.error('{:s} not exist and has been made'.format(save_dir))
    os.makedirs(save_dir)

if is_batch == False:
    # test hnet model on single image
    test_lanenet(image_path, weights_path, use_gpu)
else:
    # test hnet model on a batch of image
    test_lanenet_batch(image_dir=image_path, weights_path=weights_path,
                       save_dir=save_dir, use_gpu=use_gpu, batch_size=batch_size)

I0307 11:06:48.793189 81 <ipython-input-6-e66f42ab24e2>:16] Start getting the image file path...


INFO:tensorflow:Restoring parameters from /tf/network/lanenet-lane-detection/weights/tusimple_lanenet_vgg_2018-10-19-13-33-56.ckpt-200000


I0307 11:06:51.411983 81 tf_logging.py:115] Restoring parameters from /tf/network/lanenet-lane-detection/weights/tusimple_lanenet_vgg_2018-10-19-13-33-56.ckpt-200000
I0307 11:06:51.466737 81 <ipython-input-6-e66f42ab24e2>:51] [Epoch:0] Start image reading and preprocessing...
I0307 11:06:51.515768 81 <ipython-input-6-e66f42ab24e2>:63] [Epoch:0] Pre-processing 5 images, total time consuming: 0.04847s,                     Average time per sheet: 0.00969
I0307 11:06:54.265454 81 <ipython-input-6-e66f42ab24e2>:71] [Epoch:0] Predicting 5 image lane lines, total time: 2.74876s,                     average per time: 0.54975s
I0307 11:06:54.460437 81 <ipython-input-6-e66f42ab24e2>:102] [Epoch:0] Perform 5 image lane line clustering, total time: 0.15694s,                     average time per sheet: 0.03139
